In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
from researchlib.single_import import *

In [18]:
train_data = FromDataset(MNIST(True), batch_size=256)
test_data = FromDataset(MNIST(False), batch_size=256)

In [20]:
class Auxiliary(nn.Module):
    def __init__(self, f):
        super().__init__()
        self.f = f
        self.store = None
    
    def forward(self, x):
        # Store the result
        # Return the origin value
        self.store = self.f(x)
        return x 

In [34]:
model = builder([
    nn.Conv2d(1, 20, 5, 1),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),
    nn.Conv2d(20, 50, 5, 1),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),
    Flatten(),
    nn.Linear(4*4*50, 500),
    nn.ReLU(),
    nn.Linear(500, 10),
    Auxiliary(nn.LogSoftmax(1)),
    nn.LogSoftmax(1)
])

In [35]:
runner = Runner(model, train_data, test_data, 'adam', ['nll', 'nll'])

In [38]:
runner.fit(2)

  Epochs    train_loss    train_acc      val_loss      val_acc    
    1         0.0630        0.9899        0.0640        0.9903    


    2         0.0508        0.9921        0.0495        0.9915    
